## 2. FeatureExtraction_ORB

## Run name

In [1]:
import time

project_name = 'Google_LandMark_Rec'
step_name = 'FeatureExtraction_ORB'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google_LandMark_Rec_FeatureExtraction_ORB_20180510_092824


## Import PKGs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import math
import shutil
import zipfile
import pickle
import h5py
import cv2
from PIL import Image

from tqdm import tqdm
import multiprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

cpu_amount = multiprocessing.cpu_count()
print('cpu_amount: ', cpu_amount)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


cpu_amount:  4


## Project folders

In [3]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## Preview csv

In [4]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

train_csv.shape is (1225029, 3).


,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651


test_csv.shape is (117703, 2).


,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...


In [5]:
train_id = train_csv['id']
train_landmark_id = train_csv['landmark_id']
print('len(train_landmark_id) = \t%s' % len(list(set(train_landmark_id))))

id_2_landmark_id_dict = dict(zip(train_id, train_landmark_id))
print('len(id_2_landmark_id_dict) = \t%d' % len(id_2_landmark_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))

len(train_landmark_id) = 	14951
len(id_2_landmark_id_dict) = 	1225029
id: cacf8152e2d2ae60, 	landmark_id:4676
id: 0a58358a2afd3e4e, 	landmark_id:6651


## FeatureExtraction_ORB

In [6]:
def image_detect_and_compute(image_file, clf):
    """Detect and compute interest points and their descriptors."""
    img = cv2.imread(image_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, des = clf.detectAndCompute(img, None)
    return des

n_features = 500
clf = cv2.ORB_create(n_features)

org_train_images = os.listdir(org_train_folder)[:10]
print(len(org_train_images))
image_file = os.path.join(org_train_folder, org_train_images[0])
print(image_file)

10
/data1/kaggle/landmark-recognition-challenge/input/org_train/69b846bd58c3f09a.jpg


### Basic process

In [7]:
des = image_detect_and_compute(image_file, clf)
print(type(des))
print(des.shape)
print('%.2f Mb' % (sys.getsizeof(des)/1024/1024))

<class 'numpy.ndarray'>
(500, 32)
0.02 Mb


In [8]:
image_features = {}
n_rows = 10
for i, image_name in enumerate(os.listdir(org_train_folder)[:n_rows]):
    image_id = image_name[:-4]
    print(image_name, image_id)
    image_file = os.path.join(org_train_folder, image_name)
    des = image_detect_and_compute(image_file, clf)
    print(des.shape)
    print(des[0][:15])
    image_features[image_id] = des

print('*'*80)
print('len_image_features=', len(image_features.keys()))
for i, image_id in enumerate(image_features.keys()):
    print('image_id: %s, \tlandmark_id:%s,\t feature_shape: \t' % (image_id, id_2_landmark_id_dict[image_id]), image_features[image_id].shape)


69b846bd58c3f09a.jpg 69b846bd58c3f09a
(500, 32)
[128 230 157 251 108 201 109 205  86 119  69 146 183 195 250]
19a1de4f08cd0305.jpg 19a1de4f08cd0305
(500, 32)
[139 120 170 191  44 191 187 208  30 235  87 183 253 252 243]
4ee821754ef5fd83.jpg 4ee821754ef5fd83
(500, 32)
[148 167   6 171 154  11 110 105  46  49 227 210 197  67   8]
5f7f65b33582323e.jpg 5f7f65b33582323e
(495, 32)
[148 221  91 175 148 199 103 253  20   0  91  66 119 119  40]
c1943ca95701cbc7.jpg c1943ca95701cbc7
(500, 32)
[247 164 132 147 137  61 136   9 139 241 141 220 194 236  94]
96f38f15ea1a0731.jpg 96f38f15ea1a0731
(500, 32)
[184  73 118 253  21 214  86 244  85  12 242   3 107  25  53]
1a4b8561cb5f267b.jpg 1a4b8561cb5f267b
(500, 32)
[252 188 254 122 171 249  85 222 243  31 164  56  59 255  66]
2a85b4b43c7f7c83.jpg 2a85b4b43c7f7c83
(500, 32)
[ 41 169  51  91  77 221 224 197 239  73 228   8 162 152  86]
d18895737fe52d04.jpg d18895737fe52d04
(500, 32)
[216 156  21 248 139 228  17 166 164  23 130  56 159  55   1]
af89b3a142

In [9]:
%%time
image_features_file = os.path.join(feature_folder, 'feature_%s.pickle' % run_name)
print(image_features_file)
# pickle.dump(image_features, open(image_features_file, "wb"), True)
# image_features = pickle.load(open(image_features_file, "rb"))

print('len_image_features=', len(image_features.keys()))
for i, image_id in enumerate(image_features.keys()):
    print('image_id: %s, \tlandmark_id:%s,\t feature_shape: \t' % (image_id, id_2_landmark_id_dict[image_id]), image_features[image_id].shape)


/data1/kaggle/landmark-recognition-challenge/feature/feature_Google_LandMark_Rec_FeatureExtraction_ORB_20180510_092824.pickle
len_image_features= 10
image_id: 69b846bd58c3f09a, 	landmark_id:6051,	 feature_shape: 	 (500, 32)
image_id: 19a1de4f08cd0305, 	landmark_id:9179,	 feature_shape: 	 (500, 32)
image_id: 4ee821754ef5fd83, 	landmark_id:11301,	 feature_shape: 	 (500, 32)
image_id: 5f7f65b33582323e, 	landmark_id:9633,	 feature_shape: 	 (495, 32)
image_id: c1943ca95701cbc7, 	landmark_id:9119,	 feature_shape: 	 (500, 32)
image_id: 96f38f15ea1a0731, 	landmark_id:9779,	 feature_shape: 	 (500, 32)
image_id: 1a4b8561cb5f267b, 	landmark_id:10914,	 feature_shape: 	 (500, 32)
image_id: 2a85b4b43c7f7c83, 	landmark_id:2949,	 feature_shape: 	 (500, 32)
image_id: d18895737fe52d04, 	landmark_id:13526,	 feature_shape: 	 (500, 32)
image_id: af89b3a1428973b4, 	landmark_id:4734,	 feature_shape: 	 (500, 32)
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 727 µs


In [10]:
%%time
def dump_pickle_feature(run_name, image_features):
    image_features_file = os.path.join(feature_folder, 'feature_%s.pickle' % run_name)
    print('Dump: ', image_features_file, end='  ')
    print(len(image_features.keys()))
    pickle.dump(image_features, open(image_features_file, "wb"), True)

def load_pickle_feature(run_name):
    image_features_file = os.path.join(feature_folder, 'feature_%s.pickle' % run_name)
    image_features = pickle.load(open(image_features_file, "rb"))
    print('Load: ', image_features_file, end='  ')
    print(len(image_features.keys()))
    return image_features

# dump_pickle_feature(run_name, image_features)
# image_features = load_pickle_feature(run_name)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.25 µs


## Official code

In [11]:
image_features = {}
n_rows = 5*10000
for i, image_name in enumerate(os.listdir(org_train_folder)[:n_rows]):
    image_id = image_name[:-4]
    image_file = os.path.join(org_train_folder, image_name)
    des = image_detect_and_compute(image_file, clf)
    image_features[image_id] = des
    if i < 3:
        print(image_name, image_id, end=' ')
        print(des.shape, end=' ')
        print(des[0][:10])
    if (i+1) % 1000 == 0:
        print(int((i+1)/1000), end=' ')
        

print('*'*80)
print('len_image_features=', len(image_features.keys()))
for i, image_id in enumerate(list(image_features.keys())[:3]):
    print('image_id: %s, \tlandmark_id:%s,\t feature_shape: ' % (image_id, id_2_landmark_id_dict[image_id]), image_features[image_id].shape)


69b846bd58c3f09a.jpg 69b846bd58c3f09a (500, 32) [128 230 157 251 108 201 109 205  86 119]
19a1de4f08cd0305.jpg 19a1de4f08cd0305 (500, 32) [139 120 170 191  44 191 187 208  30 235]
4ee821754ef5fd83.jpg 4ee821754ef5fd83 (500, 32) [148 167   6 171 154  11 110 105  46  49]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 ********************************************************************************
len_image_features= 50000
image_id: 69b846bd58c3f09a, 	landmark_id:6051,	 feature_shape:  (500, 32)
image_id: 19a1de4f08cd0305, 	landmark_id:9179,	 feature_shape:  (500, 32)
image_id: 4ee821754ef5fd83, 	landmark_id:11301,	 feature_shape:  (500, 32)


In [12]:
%%time
dump_pickle_feature(run_name, image_features)
image_features = load_pickle_feature(run_name)

Dump:  /data1/kaggle/landmark-recognition-challenge/feature/feature_Google_LandMark_Rec_FeatureExtraction_ORB_20180510_092824.pickle  50000
Load:  /data1/kaggle/landmark-recognition-challenge/feature/feature_Google_LandMark_Rec_FeatureExtraction_ORB_20180510_092824.pickle  50000
CPU times: user 9.21 s, sys: 3.16 s, total: 12.4 s
Wall time: 30 s


In [13]:
print('*'*80)
print('len_image_features=', len(image_features.keys()))
for i, image_id in enumerate(list(image_features.keys())[:3]):
    print('image_id: %s, \tlandmark_id:%s,\t feature_shape: ' % (image_id, id_2_landmark_id_dict[image_id]), image_features[image_id].shape, end=' ')
    print(image_features[image_id][0][:10])

********************************************************************************
len_image_features= 50000
image_id: 69b846bd58c3f09a, 	landmark_id:6051,	 feature_shape:  (500, 32) [128 230 157 251 108 201 109 205  86 119]
image_id: 19a1de4f08cd0305, 	landmark_id:9179,	 feature_shape:  (500, 32) [139 120 170 191  44 191 187 208  30 235]
image_id: 4ee821754ef5fd83, 	landmark_id:11301,	 feature_shape:  (500, 32) [148 167   6 171 154  11 110 105  46  49]
